---
# Projeto - Previsão de preço de casas no DF
---

#### **Machine Learning em Projetos (_Pablo Alves, Filipe Basílico e Carlos Roberto_)**


---
### **Descrição do projeto de machine learning**

Antes de escrever qualquer código, precisamos entender o problema que queremos resolver e fazer uma descrição eficiente do projeto, visando a comunicação simples e rápida do objetivo do projeto para técnicos, gestores e colaboradores.

**1- Nome do projeto (apelido)**

- Previsão de preço de casas no DF

**2- Descrição do problema ou tarefa:**

- Ao se deparar com uma casa em um anúncio, confirmar se o seu preço é justo

**3- Descrição da solução de IA:**

- Treinamento supervisionado de modelo para realizar a previsão do valor de uma casa situada no Distrito Federal.

**4- Fonte de dados:**

- Webscraping de sites tipo olx, dfimoveis e similares.
- link: https://www.kaggle.com/datasets/devvret/brazil-real-estate-listings

**5- Variáveis independentes (preditoras ou "features"):**

- Localização, metragem, quantidade de quartos, vagas de garagem e banheiros.

**6- Variável dependente (resposta ou "target"):**

- Preço do anúncio
---

Data: 10/05/2023

Autor

- Pablo Alves - TST - pablo.alves@tst.jus.br

Participante:

- Fillipe Ballico de Moraes - ORGAO - EMAIL
- Carlos Roberto - SSPAL - carlos.silva@pm.al.gov.br // tenroberto2011@gmail.com

---


### Carregar bibliotecas de código


In [9]:
# # Instalando às Libs
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
!pip install seaborn
# !pip install warnings
!pip install catboost
!pip install lightgbm
!pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 2.8 MB/s eta 0:00:0000:0100:010m
  Using cached wheel-0.40.0-py3-none-any.whl (64 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 3.0 MB/s eta 0:00:0000:0100:010m


In [10]:
# Importar bibliotecas (pandas etc)
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", None)
pd.set_option("display.float_format", lambda x: "%.3f" % x)

## 1- Análise exploratória do dados


In [12]:
# Ler dados
# arq_dados = "./dados/properati-BR-2016-11-01-properties-sell.csv"
arq_dados = "./dados/properati_br_2016_11_01_properties_rent.csv"

dados = pd.read_csv(
    arq_dados,
)

# df.to_csv("./dados/precos_br_2016.csv", index=False)
# df_all
dados.head()

,created_on,operation,property_type,place_name,place_with_parent_names,geonames_id,lat_lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,location
0,2013-04-25,rent,apartment,Praia Grande,|Brasil|São Paulo|Praia Grande|,NaN,NaN,NaN,NaN,900.000,BRL,898.290,280.840,NaN,NaN,NaN,NaN,NaN,1.000,NaN,http://praia-grande-sao-paulo.properati.com.br...,"Otimo Imovel com o melhor valor da regiao, con...",Apartamento Em Praia Grande,NaN,NaN
1,2013-04-25,rent,apartment,Alphaville,|Brasil|São Paulo|Barueri|Alphaville|,NaN,NaN,NaN,NaN,3200.000,BRL,3193.990,998.560,NaN,NaN,NaN,NaN,NaN,3.000,NaN,http://alphaville-barueri.properati.com.br/q8z...,"Otimo Imovel com o melhor valor da regiao, con...",Apartamento Em Barueri,https://thumbs-cf.properati.com/0/Wjq7toq-DpwK...,NaN
2,2013-04-25,rent,house,Centro,|Brasil|São Paulo|Jundiaí|Centro|,NaN,NaN,NaN,NaN,1800.000,BRL,1796.620,561.690,NaN,NaN,NaN,NaN,NaN,2.000,NaN,http://centro-jundiai.properati.com.br/qe2_alu...,"Otimo Imovel com o melhor valor da regiao, con...",Casa Em Jundiai,NaN,NaN
3,2013-04-25,rent,apartment,Alphaville,|Brasil|São Paulo|Barueri|Alphaville|,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://alphaville-barueri.properati.com.br/s9p...,"Otimo Imovel com o melhor valor da regiao, con...",Apartamento Em Barueri,https://thumbs-cf.properati.com/0/X7PtsZjj8Vqz...,NaN
4,2013-04-25,rent,apartment,Barueri,|Brasil|São Paulo|Barueri|,NaN,NaN,NaN,NaN,1800.000,BRL,1796.620,561.690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://barueri.properati.com.br/s9u_aluga-se_a...,"Otimo Imovel com o melhor valor da regiao, con...",Apartamento Em Barueri,https://thumbs-cf.properati.com/9/URn3U3gg-F6C...,NaN


In [13]:
dados.shape

(97353, 25)

In [14]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97353 entries, 0 to 97352
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   created_on                  97353 non-null  object 
 1   operation                   97353 non-null  object 
 2   property_type               97353 non-null  object 
 3   place_name                  97353 non-null  object 
 4   place_with_parent_names     97353 non-null  object 
 5   geonames_id                 3 non-null      float64
 6   lat_lon                     46030 non-null  object 
 7   lat                         46030 non-null  float64
 8   lon                         46030 non-null  float64
 9   price                       92334 non-null  float64
 10  currency                    92333 non-null  object 
 11  price_aprox_local_currency  92334 non-null  float64
 12  price_aprox_usd             92334 non-null  float64
 13  surface_total_in_m2         158

In [15]:
dados.columns

Index(['created_on', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'geonames_id', 'lat_lon', 'lat', 'lon',
       'price', 'currency', 'price_aprox_local_currency', 'price_aprox_usd',
       'surface_total_in_m2', 'surface_covered_in_m2', 'price_usd_per_m2',
       'price_per_m2', 'floor', 'rooms', 'expenses', 'properati_url',
       'description', 'title', 'image_thumbnail', 'location'],
      dtype='object')

In [16]:
# apenas dados da imoveis em Brasília
# dados_df = dados.loc[dados['place_with_parent_names'] == {'Brasil','Distrito Federal','Brasília','' }]
dados_df = dados.loc[dados["place_name"] == "Brasília"]
dados_df.to_csv("./dados/dados_df.csv")

In [17]:
dados_df.shape

(63, 25)

In [106]:
dados_df.head()

,created_on,operation,property_type,place_name,place_with_parent_names,geonames_id,lat-lon,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
13400,2014-01-09,sell,apartment,Brasília,|Brasil|Distrito Federal|Brasília|,NaN,"-15.7942287,-47.8821658",-15.794,-47.882,388194.000,BRL,397438.460,124253.880,NaN,34.000,3654.526,11417.471,NaN,1.000,NaN,http://brasilia-distrito-federal.properati.com...,APARTAMENTO DE 1 QUARTO NOROESTE - ATRIUM PLAT...,ATRIUM PLATINE NOROESTE-APARTAMENTO DE 1 QUARTO,https://thumbs-cf.properati.com/2/C5v3t9lgl1Wn...
13401,2014-01-09,sell,apartment,Brasília,|Brasil|Distrito Federal|Brasília|,NaN,"-15.7942287,-47.8821658",-15.794,-47.882,736200.000,BRL,753731.920,235644.320,NaN,87.000,2708.555,8462.069,NaN,1.000,NaN,http://brasilia-distrito-federal.properati.com...,EMPREENDIMENTO: - Atrium D'Argent. Excelente ...,COBERTURA DUPLEX 1 QUARTO NOROESTE - Atrium D'...,https://thumbs-cf.properati.com/3/xGvaV9zPuh9Z...
13402,2014-01-09,sell,apartment,Brasília,|Brasil|Distrito Federal|Brasília|,NaN,"-15.7942287,-47.8821658",-15.794,-47.882,804492.000,BRL,823650.240,257503.360,NaN,96.000,2682.327,8380.125,NaN,3.000,NaN,http://brasilia-distrito-federal.properati.com...,Bossa Nova Noroeste. Conheça excelente Apartam...,BOSSA NOVA NOROESTE-APARTAMENTO 3 QUARTOS,https://thumbs-cf.properati.com/7/16jzon_13yOT...
13403,2014-01-09,sell,apartment,Brasília,|Brasil|Distrito Federal|Brasília|,NaN,"-15.8729537,-48.0163953",-15.873,-48.016,287229.000,BRL,294069.080,91936.810,NaN,62.000,1482.852,4632.726,NaN,2.000,NaN,http://brasilia-distrito-federal.properati.com...,VIVACE SHOW DE MORAR TAGUATINGA-APARTAMENTO 2 ...,Vivace Show de Morar Taguatinga-Apartamento 2 ...,https://thumbs-cf.properati.com/8/brmgqVzyE3Gj...
13404,2014-01-09,sell,apartment,Brasília,|Brasil|Distrito Federal|Brasília|,NaN,"-15.7941454,-47.8825479",-15.794,-47.883,1176000.000,BRL,1204005.360,376416.360,NaN,127.000,2963.908,9259.843,NaN,3.000,NaN,http://brasilia-distrito-federal.properati.com...,PIGOT RESIDENCIAL: Conheça este excelente Apar...,APARTAMENTO 3 QUARTOS NOROESTE - Pigot Residen...,https://thumbs-cf.properati.com/1/oFTj_4cqx1Na...


In [4]:
# Função para checar o dataset e ajustar pelos tipos
def check_df(dataframe, head=10):
    print(
        "############################################## SHAPE #############################################"
    )
    print(dataframe.shape)
    print(
        "############################################## Types #############################################"
    )
    print(dataframe.dtypes)
    print(
        "############################################# # Head #############################################"
    )
    print(dataframe.head(head))
    print(
        "############################################## Tail ##############################################"
    )
    print(dataframe.tail(head))
    print(
        "############################################# NA #################################################"
    )
    print(dataframe.isnull().sum())
    print(
        "########################################## Quantiles ############################################# "
    )
    print(dataframe.quantile([]).T)
    print(
        "##########################################  NUMBERUNIQE ########################################## "
    )
    print(dataframe.nunique())

In [102]:
check_df(dados_df)

############################################## SHAPE #############################################
(407, 24)
############################################## Types #############################################
created_on                     object
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  o

In [94]:
def grab_col_names(dataframe, cat_th=7, car_th=20):
    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [
        col
        for col in dataframe.columns
        if dataframe[col].nunique() < cat_th and dataframe[col].dtypes != "O"
    ]
    cat_but_car = [
        col
        for col in dataframe.columns
        if dataframe[col].nunique() > car_th and dataframe[col].dtypes == "O"
    ]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f"cat_cols: {len(cat_cols)}")
    print(f"num_cols: {len(num_cols)}")
    print(f"cat_but_car: {len(cat_but_car)}")
    print(f"num_but_cat: {len(num_but_cat)}")
    return cat_cols, num_cols, cat_but_car

In [53]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)

Observations: 2
Variables: 135
cat_cols: 135
num_cols: 0
cat_but_car: 0
num_but_cat: 31


In [111]:
# a
dados_df.shape

(407, 24)

In [110]:
# Analisar dimensões do dataset
df = pd.DataFrame(dados_df, columns=dados.columns)
df.head()
df.to_csv("dados_df.txt")

In [ ]:
# mostrar as primeiras linhas do dataset

In [ ]:
# Criar gráficos estatísticos

## 2- Preparação dos dados


In [123]:
# importanto as libs
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# from sklearn.ensemble import RandomForestClassifier

In [124]:
feature_names = [i for i in df.columns if df[i].dtype in [np.int64]]

# atribuir os valores das variáveis independentes a 'X'
X = df[feature_names]

# atribuir os valores da variável dependente a 'y'
y = df["price"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=1
)

my_model = RandomForestClassifier(n_estimators=100, random_state=0).fit(
    X_train, y_train
)

ValueError: at least one array or dtype is required

In [116]:
# Dividir o dataset entre treino e teste (85/15)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 0, stratify = y)
print("Dimensões de X_train = ", X_train.shape)
print("Dimensões de X_test = ", X_test.shape)

Dimensões de X_train =  (325, 0)
Dimensões de X_test =  (82, 0)


### 2.1- Codificação das variáveis independentes


#### 2.1.1- Pandas Get Dummies


In [ ]:
# df.head()

In [ ]:
# Codificando com get_dummies
# df_getdummies = pd.get_dummies(X_train)
# print('Dimensões do dataset com get_dummies: ', df_getdummies.shape)
# df_getdummies.head()

#### 2.1.2- One-hot encoder do Sklearn


In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# onehotenc = OneHotEncoder(handle_unknown='ignore', sparse=False)

In [ ]:
# onehotenc.fit(X_train)

In [ ]:
# onehotenc.categories_

In [ ]:
# df_onehotenc = pd.DataFrame(onehotenc.fit_transform(X_train),columns=onehotenc.get_feature_names_out())
# print('Dimensões do dataset com sklearn onehotenc: ', df_onehotenc.shape)
# df_onehotenc.head()

#### 2.1.3- Ordinal Encoder


In [ ]:
# Codificando com OrdinalEncoder
# from sklearn.preprocessing import OrdinalEncoder
# ordenc = OrdinalEncoder()

In [ ]:
# df_ordenc = pd.DataFrame(ordenc.fit_transform(X_train))
# df_ordenc.head()

In [ ]:
df_ordenc.columns = ["buying", "maint", "doors", "persons", "lug_boot", "safety"]
print("Dimensões do dataset com OrdinalEncoder: ", df_ordenc.shape)
df_ordenc.head()

Dimensões do dataset com OrdinalEncoder:  (1468, 6)


,buying,maint,doors,persons,lug_boot,safety
0,0.0,1.0,2.0,0.0,0.0,0.0
1,2.0,0.0,1.0,0.0,1.0,0.0
2,2.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,3.0,0.0,0.0,2.0
4,1.0,1.0,2.0,2.0,0.0,0.0


In [ ]:
ordenc.categories_

[array(['high', 'low', 'med', 'vhigh'], dtype=object),
 array(['high', 'low', 'med', 'vhigh'], dtype=object),
 array(['2', '3', '4', '5more'], dtype=object),
 array(['2', '4', 'more'], dtype=object),
 array(['big', 'med', 'small'], dtype=object),
 array(['high', 'low', 'med'], dtype=object)]

Conclusões sobre o Ordinal Encoder :

- O ordinal encoder utilizou a ordem lexicográfica para codificação dos valores das variáveis, resultando numa péssima conversão. Isto irá prejudicar o desempenho do modelo.  
  Por exemplo:  
  "safety" : high=0, low=1 e med=2.  
  o correto seria: low=0, med=1 e high=2.
- Sempre confira o resultado, principalmente, do Ordinal encoder!


#### 2.1.4- Manual Enconding


In [ ]:
# Mostra os valores únicos de cada variável para (fazer loop)
for col in df:
    print(col, ":", df[col].unique())

buying : ['vhigh' 'high' 'med' 'low']
maint : ['vhigh' 'high' 'med' 'low']
doors : ['2' '3' '4' '5more']
persons : ['2' '4' 'more']
lug_boot : ['small' 'med' 'big']
safety : ['low' 'med' 'high']
car : ['unacc' 'acc' 'vgood' 'good']


In [ ]:
# A ordem das categorias não foi respeitada no Ordinal encoder acima.
# Façamos um encoding manual, respeitando essas ordens relativas.
# Criar dicionário para codificar os dados das colunas com o comando replace
encoding = {
    "low": 0,
    "small": 0,
    "med": 1,
    "high": 2,
    "big": 2,
    "2": 0,
    "3": 1,
    "4": 2,
    "more": 3,
    "5more": 3,
    "vhigh": 3,
}

In [ ]:
dic_persons = {"2": 0, "4": 1, "more": 2}
df_manualenc = pd.DataFrame(X.persons.replace(dic_persons))

In [ ]:
# Mostre as primeiras linhas de X para poder visualizar a diferença quando executar o comando na célula abaixo
X.head()

,buying,maint,doors,persons,lug_boot,safety
0,vhigh,vhigh,2,2,small,low
1,vhigh,vhigh,2,2,small,med
2,vhigh,vhigh,2,2,small,high
3,vhigh,vhigh,2,2,med,low
4,vhigh,vhigh,2,2,med,med


In [ ]:
# Criar um novo dataframe com a codificação do Manual encoding (usando replace) e mostrar as primeiras linhas e dimensões
df_manualenc = pd.DataFrame(X.replace(encoding))
df_manualenc.head()

,buying,maint,doors,persons,lug_boot,safety
0,3,3,0,0,0,0
1,3,3,0,0,0,1
2,3,3,0,0,0,2
3,3,3,0,0,1,0
4,3,3,0,0,1,1


In [ ]:
# Dividir o dataset entre treino e teste

## 3 - Escolha do Modelo


## 4 - Treinar o modelo


In [ ]:
# Entre código aqui...

# instanciar modelo


# treinar modelo com dados de treino

## 5- Testar o(s) modelo(s)


In [ ]:
# fazer predição da espécie para uma nova flor com as dimensões: [5.1, 3.5, 1.4, 0.2]

In [ ]:
# mostrar resultado da predição

## 6- Avaliar o modelo


In [ ]:
# mostrar score

In [ ]:
# mostrar matrix de confusão ou Curva ROC

## Conclusão

No nosso experimento, treinamos modelos de regressão logística sobre os dados de aceitabilidade de carros com diferentes codificações, gerando os seguintes resultados:

- pandas get_dummies: 90,3% (f1-score)
- one-hot encoding: 91,7% (f1-score)
- ordinal encoding (sem seguir a ordem das categorias): 61,5% (f1-score)
- manual encoding (substituição), seguindo a ordem relativa das categorias: 80,0% (f1-score)
